#### Question 1: Running Elastic

In [40]:
from elasticsearch import Elasticsearch

In [41]:
# Instatiate the ES Client
es_client = Elasticsearch('http://localhost:9200')

In [42]:
es_info = es_client.info()
build_hash = es_info['version']['build_hash']
print('ES build hash:',build_hash)

ES build hash: dbcbbbd0bc4924cfeb28929dc05d82d662c527b7


#### Question 2. Indexing the data

In [43]:
%run Elastic_Indexes_Setup.ipynb

  0%|          | 0/948 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

In [ ]:
# index_name = "course-questions"

### Questions on ES Queries

In [ ]:
## Sample Es Query
def elastic_search_rag(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

#### Question 3: Searching

In [44]:
def q3_elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [45]:
q3_query = 'How do execute a command on a Kubernetes pod?'

In [47]:
print(q3_elastic_search(q3_query)[0])

44.50556


#### Question 4: Filtering

In [48]:
def q4_elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [49]:
q4_query="How do copy a file to a Docker container?"

In [50]:
print(q4_elastic_search(q4_query)[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


#### Question 5. Building a prompt

In [51]:
def q5_build_prompt(query, search_results):
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context =  context + context_template.format(question=doc['question'], text=doc['text'])+'\n\n'

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [52]:
q5_query="How do copy a file to a Docker container?"

In [53]:
search_results = q4_elastic_search(q5_query)

In [54]:
prompt = q5_build_prompt(q5_query, search_results)

In [55]:
# prompt
len(prompt)

1446

#### Question 6. Tokens

In [56]:
import tiktoken

In [57]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [58]:
token=encoding.encode(prompt)

In [59]:
len(token)

320

#### RAG

In [66]:
from openai import OpenAI
client = OpenAI()

In [67]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [68]:
def q5_rag(query):
    search_results = q4_elastic_search(query)
    prompt = q5_build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [70]:
llm_response = q5_rag(q5_query)

In [71]:
llm_response

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\n\nThis command allows you to copy files from your local machine to a running Docker container.'

In [72]:
len(llm_response)

288